capture audio and video data

In [1]:
import cv2 as cv
import pyaudio
import wave
import time
import threading

def record_video_and_audio():
    cap = cv.VideoCapture(0)
    fourcc = cv.VideoWriter_fourcc(*'mp4v')
    out = cv.VideoWriter('C:\\Users\\chels\\Desktop\\COSC490-MER\\outputs\\video\\output.mp4', fourcc, 20.0, (640, 480))

    CHUNK = 4096  # Increased chunk size for smoother audio
    FORMAT = pyaudio.paInt16
    CHANNELS = 1
    RATE = 44100
    WAVE_OUTPUT_FILENAME = "C:\\Users\\chels\\Desktop\\COSC490-MER\\outputs\\audio\\output.wav"
    frames = []

    p = pyaudio.PyAudio()
    stream = None

    audio_started = False
    audio_running = True

    def audio_record():
        nonlocal stream, frames, audio_running
        stream = p.open(format=FORMAT,
                        channels=CHANNELS,
                        rate=RATE,
                        input=True,
                        frames_per_buffer=CHUNK)
        print("* recording audio")
        while audio_running:
            data = stream.read(CHUNK, exception_on_overflow=False)
            frames.append(data)

    audio_thread = threading.Thread(target=audio_record)
    audio_thread.start()

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            print("Can't receive frame (stream end?). Exiting ...")
            break
        out.write(frame)
        cv.imshow('frame', frame)

        if cv.waitKey(1) == ord('q'):
            break

    # Cleanup video
    cap.release()
    out.release()
    cv.destroyAllWindows()

    # Cleanup audio
    audio_running = False
    audio_thread.join()
    stream.stop_stream()
    stream.close()
    p.terminate()

    wf = wave.open(WAVE_OUTPUT_FILENAME, 'wb')
    wf.setnchannels(CHANNELS)
    wf.setsampwidth(p.get_sample_size(FORMAT))
    wf.setframerate(RATE)
    wf.writeframes(b''.join(frames))
    wf.close()
    print("* done recording audio")

grab the transcript

In [2]:
import requests
import time

def transcribe_audio():
  base_url = "https://api.assemblyai.com"

  headers = {
      "authorization": "6a35340cac1c443e8e4bbc1d027a3ad5"
  }
  with open("outputs/audio/output.wav", "rb") as f:
    response = requests.post(base_url + "/v2/upload",
                            headers=headers,
                            data=f)

  audio_url = response.json()["upload_url"]


  data = {
      "audio_url": audio_url,
      "speech_model": "universal"
  }

  url = base_url + "/v2/transcript"
  response = requests.post(url, json=data, headers=headers)

  transcript_id = response.json()['id']
  polling_endpoint = base_url + "/v2/transcript/" + transcript_id

  while True:
    transcription_result = requests.get(polling_endpoint, headers=headers).json()
    transcript_text = transcription_result['text']

    if transcription_result['status'] == 'completed':
      print("Transcription completed.")
      return transcript_text
      # with open("transcript.txt", "w") as file:
      #   file.write(transcript_text) # we don't necessarily need to write it into a file
      # file.close()

    elif transcription_result['status'] == 'error':
      raise RuntimeError(f"Transcription failed: {transcription_result['error']}")

    else:
      time.sleep(3)

sentiment analysis

In [3]:
import nltk
import pandas as pd
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from nltk.sentiment.util import *
# analyzer = SentimentIntensityAnalyzer()

# def tokens_to_text(tokens):
#     text = " ".join(tokens)
#     score = analyzer.polarity_scores(text)
#     return score

def determine_sentiment(text):
    analyzer = SentimentIntensityAnalyzer()
    score = analyzer.polarity_scores(text)
    if score['compound'] >= 0.05:
        return 'positive'
    elif score['compound'] <= -0.05:
        return 'negative'
    else:
        return 'neutral'
    

video analysis

In [4]:
from fer.fer import FER
from fer.classes import Video
import pandas as pd
def analyze_video_emotions():
    emotion_detector = FER(mtcnn=True)
    video = Video("C:\\Users\\chels\\Desktop\\COSC490-MER\\outputs\\video\\output.mp4")


    emotions = video.analyze(emotion_detector, display=False, frequency=15)
    df = pd.DataFrame(emotions)


    stats = df.describe()
    emotions = stats.loc['mean'].nlargest(2).index.tolist()
    return emotions


c:\Users\chels\Desktop\COSC490-MER\.venv\Lib\site-packages\fer\fer.py:37: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources
c:\Users\chels\Desktop\COSC490-MER\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


therapyAI chatbot response

In [5]:
from ollama import chat
from ollama import ChatResponse

def chatbot_response(emotions, sentiment, text):
  response: ChatResponse = chat(model='gemma3', messages=[
    {
      'role': 'system',
      'content': f'Based on the user\'s two primary emotions {emotions[0]} and {emotions[1]} from the video analysis, and based on the sentiment of the text which is {sentiment}, generate a therapuetic, empathetic response to the user\'s words. Avoid asking follow-up questions (if waranted), but rather provide advice as a therapist would. The user said: {text}'
    }
  ])
  # or access fields directly from the response object
  print(response.message.content)

final call

In [6]:
record_video_and_audio() # record and capture the audio
text = transcribe_audio()
sentiment = determine_sentiment(text)
emotions = analyze_video_emotions()
response = chatbot_response(emotions, sentiment, text)
print(f"TherapyAI Response: {response}\nSentiment: {sentiment}\nTranscribed Text: {text}")

* recording audio
* done recording audio
Transcription completed.


    TF 2.20. Please use the LiteRT interpreter from the ai_edge_litert package.
    See the [migration guide](https://ai.google.dev/edge/litert/migration)
    for details.
    
  warnings.warn(_INTERPRETER_DELETION_WARNING)

INFO:fer:20.00 fps, 1047 frames, 52.35 seconds
INFO:fer:Making directories at output
INFO:fer:Deleted pre-existing output\output_output.mp4
INFO:fer:Async I/O enabled for frame saving
70frames [00:09,  7.00frames/s]                    
INFO:fer:Waiting for async I/O to complete...
INFO:fer:Async I/O completed
INFO:fer:Completed analysis: saved to output\output_output.mp4
INFO:fer:Starting to Zip
INFO:fer:Compressing: 71%
INFO:fer:Zip has finished
INFO:httpx:HTTP Request: POST http://127.0.0.1:11434/api/chat "HTTP/1.1 200 OK"


It sounds like you’re carrying a significant load right now – juggling finals, student organizations, and that feeling of pressure. It's incredibly wise of you to recognize this and proactively seek ways to manage it, especially given the mix of happiness you’re feeling alongside the stress. It’s completely understandable to feel overwhelmed when so much is happening at once. 

Allowing yourself to acknowledge a decent day, even while recognizing the underlying stress, is a really important step.  Focusing on small, manageable steps – like exploring coping strategies – is a powerful way to regain a sense of control. Don’t underestimate the value of acknowledging and validating your feelings; it’s a cornerstone of healthy stress management.  

Try to carve out even just fifteen minutes a day for something that brings you a little joy, and remember, taking care of yourself during this busy time isn’t a luxury, it’s essential for your well-being and ability to navigate these challenges ef

In [7]:
emotions

['happy0', 'surprise0']